#### Restart & Run All

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 12, 15)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,23,B,SYNEX,"3,000",14.50,1,RD45pct,SET100,2022-02-02


### Print to verify before upload file

In [3]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [4]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

### Process to set target price

#### 1) Set50 records

In [5]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,22,B,BH,500,200.00,0,5pct,SET50,2022-02-02
1,2,B,CPF,4000,23.00,0,XXX,SET50,2022-02-02
2,6,B,JMART,3000,40.00,2,1L,SET50,2022-04-19
3,7,B,PTTGC,3000,41.25,0,5pct,SET50,2022-09-02
4,9,B,SCB,1000,102.00,1,XXX,SET50,2022-02-14
5,18,S,KCE,1000,52.00,0,5pct,SET50,2022-08-22
6,20,S,SCC,300,350.00,0,5pct,SET50,2022-08-10


In [6]:
name = 'BH'
limit = 20 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,20.00,20.00,20.00,20.00,20.00
mean,217.40,220.50,214.60,3938796.60,217.90
std,6.06,6.05,6.35,2692665.69,6.49
min,206.00,210.00,203.00,1378122.00,206.00
25%,212.75,216.75,210.00,2188828.25,215.25
50%,217.50,220.50,215.50,3032056.00,218.50
75%,220.00,223.75,217.25,4509320.50,221.00
max,229.00,232.00,228.00,12380014.00,230.00


#### 2) Set100 records

In [7]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,CKP,20000,4.6,1,3L,SET100,2022-02-02
1,8,B,RCL,3000,28.0,1,5pct,SET100,2022-11-24
2,10,B,STA,2500,18.1,1,5pct,SET100,2022-08-23
3,23,B,SYNEX,3000,14.5,1,RD45pct,SET100,2022-02-02
4,16,S,DOHOME,2400,14.6,1,5pct,SET100,2022-02-02


In [8]:
name = 'BCH'
limit = 60 # 3 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,18.83,19.01,18.59,15292756.58,18.84
std,1.13,1.10,1.10,11187185.13,1.11
min,16.90,17.30,16.80,4713913.00,17.20
25%,17.70,17.98,17.60,8465316.50,17.80
50%,18.80,18.90,18.55,11788496.00,18.80
75%,19.92,20.10,19.70,19974086.50,20.00
max,20.50,20.70,20.20,75617776.00,20.60


#### 3) Set records

In [9]:
# Why SIS does not print out from this SQL?
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,3,B,GFPT,7500,12.50,1,XXX,SET,2022-02-02
1,4,B,GVREIT,10000,8.90,2,C8.90,SET,2022-12-08
2,5,B,III,8000,12.50,0,XXX,SET,2022-02-02
3,12,B,TFFIF,10000,7.40,1,DOS,SET,2022-12-09
4,13,B,TFG,20000,5.00,1,XXX,SET,2022-02-02
5,14,S,ASP,10000,3.10,1,5pct,SET,2022-08-24
6,15,S,DIF,10000,14.60,0,CUT,SET,2022-11-11
7,17,S,JASIF,10000,8.80,0,CP1S,SET,2022-11-21
8,19,S,MCS,25000,9.70,0,5pct,SET,2022-09-03
9,21,S,TMT,12000,7.25,1,C9.0,SET,2022-08-24


In [10]:
name = 'TPIPL'
limit = 120 # 6 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,1.69,1.70,1.67,16798618.88,1.69
std,0.11,0.11,0.11,15438764.01,0.11
min,1.50,1.51,1.49,1669720.00,1.51
25%,1.62,1.62,1.58,7351830.00,1.60
50%,1.70,1.71,1.68,12356153.50,1.70
75%,1.76,1.78,1.74,19413931.25,1.76
max,1.91,1.92,1.88,93746128.00,1.91


In [11]:
pd.read_sql_query('SELECT * FROM orders ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,22,B,BH,500,200.00,0,5pct,SET50,2022-02-02
1,1,B,CKP,20000,4.60,1,3L,SET100,2022-02-02
2,2,B,CPF,4000,23.00,0,XXX,SET50,2022-02-02
3,3,B,GFPT,7500,12.50,1,XXX,SET,2022-02-02
4,4,B,GVREIT,10000,8.90,2,C8.90,SET,2022-12-08
5,5,B,III,8000,12.50,0,XXX,SET,2022-02-02
6,6,B,JMART,3000,40.00,2,1L,SET50,2022-04-19
7,7,B,PTTGC,3000,41.25,0,5pct,SET50,2022-09-02
8,8,B,RCL,3000,28.00,1,5pct,SET100,2022-11-24
9,9,B,SCB,1000,102.00,1,XXX,SET50,2022-02-14
